In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Step 2: Gather closing prices for the last 3 years
file_path = '/content/HistoricalData_1706021650642.csv'  # Replace with your file path
data = pd.read_csv(file_path, parse_dates=['Date'])
data.set_index('Date', inplace=True)

# Step 3: Pick at least 4 technical indicators
data['MA'] = data['Close'].rolling(window=14).mean()
data['RSI'] = 100 - (100 / (1 + (data['Close'].diff(1).apply(lambda x: 0 if x == 0 else max(0, x))).rolling(window=14).mean() /
                             (data['Close'].diff(1).apply(lambda x: 0 if x == 0 else abs(x))).rolling(window=14).mean()))
std = data['Close'].rolling(window=20).std()
data['Upper'] = data['MA'] + 2 * std
data['Lower'] = data['MA'] - 2 * std

data['EMA12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['EMA26'] = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA12'] - data['EMA26']

# Step 4: Perform Correlation analysis
correlation_matrix = data[['MA', 'RSI', 'Upper', 'Lower', 'MACD']].corr()

# Step 5: Come up with a combined indicator
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[['MA', 'RSI', 'Upper', 'Lower', 'MACD']])
weights = correlation_matrix.sum().sort_values(ascending=False)
data['Combined_Indicator'] = np.dot(scaled_data, weights.values) / weights.sum()

# Step 6: Methodology to predict bullish and bearish positions
threshold = data['Combined_Indicator'].mean()
data['Signal'] = np.where(data['Combined_Indicator'] > threshold, 'Bullish', 'Bearish')

# Step 7: Report the accuracy of prediction
data['Actual_Position'] = np.where(data['Close'].shift(-1) > data['Close'], 'Bullish', 'Bearish')
accuracy = np.mean(data['Signal'] == data['Actual_Position'])
print(f'Accuracy of Prediction: {accuracy * 100:.2f}%')

# Save the results to a new CSV file
data.to_csv('financial_analysis_results.csv')


Accuracy of Prediction: 50.64%


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Step 2: Gather closing prices for the last 3 years
file_path = '/content/HistoricalData_1706021650642.csv'  # Replace with your file path
data = pd.read_csv(file_path, parse_dates=['Date'])
data.set_index('Date', inplace=True)

# Convert 'Close' column to numeric by removing the dollar sign ('$') and converting to float
data['Close'] = data['Close'].replace('[\$,]', '', regex=True).astype(float)

# Step 3: Pick at least 4 technical indicators
data['MA'] = data['Close'].rolling(window=14).mean()
data['RSI'] = 100 - (100 / (1 + (data['Close'].diff(1).apply(lambda x: 0 if x == 0 else max(0, x))).rolling(window=14).mean() /
                             (data['Close'].diff(1).apply(lambda x: 0 if x == 0 else abs(x))).rolling(window=14).mean()))
std = data['Close'].rolling(window=20).std()
data['Upper'] = data['MA'] + 2 * std
data['Lower'] = data['MA'] - 2 * std

data['EMA12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['EMA26'] = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA12'] - data['EMA26']

# Step 4: Perform Correlation analysis
correlation_matrix = data[['MA', 'RSI', 'Upper', 'Lower', 'MACD']].corr()

# Step 5: Come up with a combined indicator
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[['MA', 'RSI', 'Upper', 'Lower', 'MACD']])
weights = correlation_matrix.sum().sort_values(ascending=False)
data['Combined_Indicator'] = np.dot(scaled_data, weights.values) / weights.sum()

# Step 6: Methodology to predict bullish and bearish positions
threshold = data['Combined_Indicator'].mean()
data['Signal'] = np.where(data['Combined_Indicator'] > threshold, 'L', 'S')

# Step 7: Report the accuracy of prediction
data['Actual_Position'] = np.where(data['Close'].shift(-1) > data['Close'], 'L', 'S')
accuracy = np.mean(data['Signal'] == data['Actual_Position'])
print(f'Accuracy of Prediction: {accuracy * 100:.2f}%')

# Save the updated DataFrame to a new CSV file
data.to_csv('financial_analysis_results_with_predictions.csv')


Accuracy of Prediction: 50.64%
